In [1]:
from __future__ import print_function, division
#Import needed packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from onnx_tf.backend import prepare
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
#from pytorch2keras.converter import pytorch_to_keras
import matplotlib.pyplot as plt
import time
import os
import copy
import csv
from tensorflow import keras
import tensorflow as tf
import pandas as pd
from datetime import datetime
from sklearn import metrics
from pathlib import Path
plt.ion()   # interactive mode

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# used for Alexnet
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform_train)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform_test)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
# define samplers for obtaining training and validation batches
#train_sampler = SubsetRandomSampler(train_idx)
#valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
     num_workers=num_workers)
#valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
#    sampler=valid_sampler, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [5]:
import onnx
import onnxruntime
import coremltools
import time

Using TensorFlow backend.


In [6]:
onnx_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/onnx/'
coreml_path = '/Volumes/Cisco/Fall2021/onnx-exchange/conversion/coremltools/'
error_path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/v2/'

In [7]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
def model_scores(y_test, test_predict):
    correct_ = np.sum(y_test == test_predict)
    accuracy  = correct_ #*100./np.sum(y_test == y_test)
    return accuracy
def get_miss_classification(y1, y2):
    miss_perc_val_original_coreml = 0
    try:
        np.testing.assert_array_equal(y1, y2)
    except Exception as e:
        miss_perc_val_original_coreml = 1
    return miss_perc_val_original_coreml #, precision2, recall2, f12

In [8]:
def to_onnx(i, x, y, batch_size):
    
    # Input to the model
    #device_reset = cuda.get_current_device()
    #device_reset.reset()
    #x.cuda()
    #print("converting for batch: ", i)
    
    #torch.random.manual_seed(42)
    #x = torch.randn(10000, 3, 32, 32, requires_grad=True)
    traced_model = torch.jit.trace(model, x)
    
    ### Original Model
    since_1 = time.time()
    torch_out = model(x)
    inference_time_original = time.time() - since_1
    y0 = torch.argmax(torch_out,1)
    accuracy_original = model_scores(to_numpy(y), to_numpy(y0))
    # ONNX Model
    
    #t_elapsed_2 = time.time() - since_1
    #since_1 = time.time()
    #onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #load_time_onnx = time.time() - since_1
    #onnx.checker.check_model(onnx_model)
    #size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    #ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    since_1 = time.time()
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
    ort_outs = ort_session.run(None, ort_inputs)
    inference_time_onnx = time.time() - since_1
    # compare ONNX Runtime and PyTorch results
    #print("\n*********\n\n")
    #time_diff = t_elapsed_0+t_elapsed_1, t_elapsed_2, t_elapsed_3
    
    ####### Mis-classification ONNX ######################################
    ort_outs_tensor = torch.from_numpy(ort_outs[0]) 
    y2 = torch.argmax(ort_outs_tensor,1)
    #y2 = to_categorical(np.argmax(ort_outs[0], 1), num_classes = 10)
    #correct_onnx = np.sum(y2 == y)
    accuracy_onnx = model_scores(to_numpy(y), to_numpy(y2))
    miss_original_onnx = get_miss_classification(to_numpy(y0), to_numpy(y2))
    
    ####### End of mis-classification ONNX ###################################### 
    
    
    ## CoreML
    
    ## Converting the coremltool
    #since_1 = time.time()
    # Using image_input in the inputs parameter:
    # Convert to Core ML using the Unified Conversion API.
    #coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=x.shape)])
    #t_conversion_time_coreml = time.time() - since_1
    #since_1 = time.time()
    #coreml_model.save(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    #t_saving_time_coreml = time.time() - since_1
    
    
    #print(name_1)

    #size_coreml = os.path.getsize(coreml_path+framework+'/{}/{}-v2.mlmodel'.format(model_short_name, model_name))
    
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    #spec = coreml_model.get_spec()
    #coreml_model = coremltools.models.MLModel(spec)
    name_1 = coreml_model.get_spec().description.input[0].name
    since_1 = time.time()
    output_dict_test = coreml_model.predict({name_1:to_numpy(x)})
    inference_time_coreml = time.time() - since_1
    ####### Mis-classification coreML ######################################
    coreml_array_output = output_dict_test[list(output_dict_test.keys())[0]]
    output_tensor = torch.from_numpy(coreml_array_output) 
    y3 = torch.argmax(output_tensor,1)
    #correct_coreml = np.sum(to_numpy(y3) == to_numpy(y))
    accuracy_coreml = model_scores(to_numpy(y3), to_numpy(y))
    miss_original_coreml = get_miss_classification(to_numpy(y0), to_numpy(y3))
    
    #print(correct_original, correct_coreml, correct_onnx, np.sum(y == y))
    ## Part 1
    
    ####### End of mis-classification coreML ######################################
    
    list_val = [accuracy_original, accuracy_onnx, accuracy_coreml, miss_original_onnx, miss_original_coreml,inference_time_original, inference_time_onnx, inference_time_coreml]
    return list_val

In [10]:
def _lets_convert(data,  data_writer_run, batch_size): # for cifar10 etc
    since = time.time()
    accuracy_original = 0.0
    accuracy_onnx = 0.0
    accuracy_coreml = 0.0
    #miss_test_original = 0
    miss_original_onnx = 0
    #miss_test_onnx = 0
    miss_original_coreml = 0
    #miss_test_coreml = 0
    
    inference_time_original = 0.0
    inference_time_onnx = 0.0
    inference_time_coreml = 0.0
    
    total_ = 0.0
    total_datasets = 0
    for i, (images, labels) in enumerate(data):
        list_val = to_onnx(i, images,labels, batch_size)
        if i%50 == 0:
            print(i, list_val)
        accuracy_original += list_val[0]
        accuracy_onnx += list_val[1]
        accuracy_coreml += list_val[2]

        miss_original_onnx += list_val[3]
        miss_original_coreml += list_val[4]

        inference_time_original += list_val[5]
        inference_time_onnx += list_val[6]
        inference_time_coreml += list_val[7]
        total_ += np.sum(to_numpy(labels) == to_numpy(labels))
        total_datasets += labels.shape[0]
        if i == 0:
            break
    time_elapsed = time.time() - since
    print('Conversion complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60) )
    data_writer_run.writerow([model_short_name,framework, training_id, model_name, round(accuracy_original*100/total_,2), round(accuracy_onnx*100/total_,2), round(accuracy_coreml*100/total_,2),  round(miss_original_onnx*100/total_datasets,2),  round(miss_original_coreml*100/total_datasets,2),'', inference_time_original/total_datasets, inference_time_onnx/total_datasets, inference_time_coreml/total_datasets, '',
                              '{:.0f}m {:.0f}s'.format((inference_time_original/total_datasets) // 60, (inference_time_original/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_onnx/total_datasets) // 60, (inference_time_onnx/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format((inference_time_coreml/total_datasets) // 60, (inference_time_coreml/total_datasets) % 60),
                              '{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)])

In [14]:
import pandas as pd 

#data_file_run = open(error_path+framework+"/{}/runtime_miss-classification_{}.csv".format(model_short_name,model_name), mode='w', newline='',
#                                  encoding='utf-8')
#data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer_run.writerow(['model','framework', 'training_id', 'model_full', "batch_size", 'round','runtime','original_load_time', 'original_infererence_time', 'runtime_load_time', 
#                          'runtime_inference_time',  'miss_classified_original_runtime_percentage','',  'encoded_miss_classified_original_runtime_percentage','encoded_miss_classified_original_test_runtime_percentage', '', 'accuracy_original', 'accuracy_runtime'])
batch_size = 10000
valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)
for i, (images, labels) in enumerate(valloader):
    images_ = images
    labels_ = labels 
    break
shape_ = images_.shape
for round_ in [4,5,6,7,8,9,10]: #,4,5,6,7,8,9,10
    training_id = round_
    model_short_name = 'resnet18'
    framework = 'pytorch'
    path = '/Volumes/Cisco/Fall2021/onnx-exchange/Training/{}/{}/'.format(framework, model_short_name)

    # Model class must be defined somewhere
    since_0 = time.time()
    model_name = 'torch_exp_resnet_2021-11-06_{}'.format(training_id)
    #model_name = 'torch_lenet5-mnist_2021-11-01_{}'.format(training_id)
    model = torch.load(path+model_name+'.pth', map_location=torch.device('cpu'))
    #resnet50_model.eval()
    t_elapsed_0 = time.time() - since_0
    size0 = os.path.getsize(path+model_name+'.pth')
    size0
    
    since_1 = time.time()
    onnx_model = onnx.load(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    load_time_onnx = time.time() - since_1
    onnx.checker.check_model(onnx_model)
    size_onnx = os.path.getsize(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    #def to_numpy(tensor):
    #    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    ort_session = onnxruntime.InferenceSession(onnx_path+framework+"/{}/{}.onnx".format(model_short_name, model_name))
    
    
    # Convert to Core ML using the Unified Conversion API.
    traced_model = torch.jit.trace(model, images_)
    coreml_model = coremltools.convert(traced_model,inputs=[coremltools.TensorType(shape=images_.shape)])
    #t_conversion_time_coreml = time.time() - since_1
    
    #since_1 = time.time()
    #coreml_model = coremltools.models.MLModel(coreml_path+framework+"/{}/{}-v2.mlmodel".format(model_short_name, model_name))
    #load_time_coreml = time.time() - since_1
    
    if not os.path.exists(error_path+framework):
        Path(error_path+framework).mkdir(parents=True, exist_ok=True)
    
    print('round: ', round_)
    flag = 0
    if not os.path.exists(error_path+framework+"/runtime_accuracy2_{}.csv".format(model_short_name)):
        data_file_run = open(error_path+framework+"/runtime_accuracy2_{}.csv".format(model_short_name), mode='w', newline='', encoding='utf-8')
    else:
        data_file_run = open(error_path+framework+"/runtime_accuracy2_{}.csv".format(model_short_name), mode='a+', newline='', encoding='utf-8')
        flag = 1
    data_writer_run = csv.writer(data_file_run, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    if flag == 0: 
        data_writer_run.writerow(['model','framework', 'training_id', 'model_full', 'accuracy_original', 'accuracy_onnx', 'accuracy_coreml', 'miss_original_onnx', 'miss_original_coreml','', 'inference_time_original', 'inference_time_onnx', 'inference_time_coreml', '', 'inference_time_original2', 'inference_time_onnx2', 'inference_time_coreml2', 'overral_time'])
    # dataloaders
    
    #valloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    #num_workers=num_workers)
    _lets_convert(valloader, data_writer_run, batch_size)
    
    data_file_run.close()

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:02<00:00, 90.61 ops/s] 


round:  4
0 [6345, 6345, 6345, 0, 0, 26.125674962997437, 15.503236055374146, 31.564836025238037]
Conversion complete in 3m 40s


Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 192/192 [00:02<00:00, 74.07 ops/s] 


round:  5


KeyboardInterrupt: 